# Corona Simulator

---

# Table of Contents

1. [System Requirements](#1.-System-Requirements)
  * [Requirements Anaylsis](#Requirements-Analysis)
  * [Visual System Mappings](#Visual-System-Mappings)
  * [Mathematical Specification](#Mathematical-Specification)


2. [System Design](#2.-System-Design)
  * [Differential Specification](#Differential-Specification)
  * [Modelling](#Modelling)
  * [Simulation](#Simulation)


3. [System Validation](#3.-System-Validation)


---

---

# 1. System Requirements

<center><img src="images/requirements.png" alt="Requirements" width="70%"/>

## Requirements Analysis

**Goal:** Simulate Corona infections

**Scope:** Get an idea of how Corona spreads. And have fun!

**Assumptions:**
- Agents interact with a certain probability (per timestep)
- Infected agents are contagious
- Infected agents can die with a certain probability
- Infected agents are healed after a certain period of time
- There is a certain chance of contagion when an uninfected agent meets an infected agent

## Visual System Mappings

#### Entity Relationship Diagram

#### Stock & Flow Diagram

## Mathematical Specification

---

---

# 2. System Design

<center><img src="images/design.png" alt="Design" width="70%"/>

## Differential Specification

## Modelling

In [ ]:
##############
# 0. IMPORTS #
##############

# CADCAD STANDARD DEPENDENCIES

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

# Included with cadCAD
import pandas as pd
import plotly.express as px

# ADDITIONAL DEPENDENCIES

# For networks
import networkx as nx

# For analytics
import numpy as np

# Visualization, analysis and plotting modules
from matplotlib import pyplot as plt
import pandas as pd
import plotly.express as px

# Standard libraries
import math
from collections import Counter

In [ ]:
###################################
# 0. INITIALIZING AGENTS NETWORKk #
###################################

# Generate a empty directional graph
G = nx.MultiDiGraph()

# Number of agents
NUMBER_AGENTS = 1000

# Percentage of the infected
INFECTION_RATE = 0.01

for agent in range(NUMBER_AGENTS):
    is_infected = np.random.rand() < INFECTION_RATE
    
    # Add agent to the graph
    G.add_node(agent, infected=is_infected, days_infected=0, alive=True)

infected = 0
uninfected = 0
for agent in G.nodes(data=True):
    if agent[1]['infected']:
        infected += 1
    else:
        uninfected += 1

print("count infected: " + str(infected))
print("count uninfected: " + str(uninfected))
# print("")
# print("agents_network:")
# print(G.nodes(data=True))

In [ ]:
######################
# 1. STATE VARIABLES #
######################

initial_state = {
    'agents_network': G
}


########################
# 2. SYSTEM PARAMETERS #
########################

system_params = {
    'interaction_probability': [0.01], # Pair-wise interaction probability
    'infection_probability': [0.1], # Infection probability
    'death_probability': [0.01], # Probability to die, if infected (per timestep)
    'days_to_heal': [6] # Days after an infected agents is healed
}


#######################
# 3. POLICY FUNCTIONS #
#######################

def p_interact(params, substep, state_history, previous_state):
    """
    Control agent interactions.
    """
    
    # Variables and parameters to be used
    G = previous_state['agents_network']
    interaction_probability = params['interaction_probability']
    infection_probability = params['infection_probability']
    
    # List of agent interactions
    interactions = []
    
    for agent_1 in G.nodes:
        for agent_2 in G.nodes:
            if agent_1 == agent_2 or G.nodes[agent_1]['infected'] == G.nodes[agent_2]['infected'] or G.nodes[agent_1]['alive'] is False or G.nodes[agent_2]['alive'] is False:
                continue
            else:
                will_interact = np.random.rand() < interaction_probability
                will_infect = np.random.rand() < infection_probability
                
                if will_interact and will_infect:
                    G.nodes[agent_1]['infected'] = True
                    G.nodes[agent_2]['infected'] = True
                    
    # Return agent interactions
    return {'interactions': interactions}


#############################
# 4. STATE UPDATE FUNCTIONS #
#############################

def s_agents_network(params, substep, state_history, previous_state, policy_input):
    # Retrieve used variables and policy inputs
    G_new = previous_state['agents_network'].copy()
    interactions = policy_input['interactions']
    
    days_to_heal = params['days_to_heal']
    death_probability = params['death_probability']
    
    for agent in G_new.nodes:
        if G_new.nodes[agent]['infected'] and G_new.nodes[agent]['alive']:
            if G_new.nodes[agent]['days_infected'] < days_to_heal:
                G_new.nodes[agent]['days_infected'] += 1
                will_die = np.random.rand() < death_probability
                if will_die:
                    G_new.nodes[agent]['alive'] = False
            else:
                G_new.nodes[agent]['days_infected'] = 0
                G_new.nodes[agent]['infected'] = False
        
    return ('agents_network', G_new)


##################################
# 5. PARTIAL STATE UPDATE BLOCKS #
##################################

# Updates run in parallel
partial_state_update_blocks = [
    {   
        'policies': {
            'p_interact': p_interact
        },
        'variables': {
            'agents_network': s_agents_network
        }
    }
]

## Simulation

In [ ]:
####################
# 6. CONFIGURATION #
####################

# the config_sim function creates an object that informs cadCAD about the scope of the simulation
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation
    "T": range(30), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

# Clear any prior configs
del configs[:]

# each cacCAD experiment has configuration options which
# we will pass to the append_configs function to create our simulation configuration
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)


################
# 7. EXECUTION #
################

exec_context = ExecutionContext() # decides how cadCAD should run a simulation (default: single or multi threaded)
simulation = Executor(exec_context=exec_context, configs=experiment.configs)
raw_result, tensor_field, sessions = simulation.execute() # execute the simulation


####################################
# 8. SIMULATION OUTPUT PREPARATION #
####################################

simulation_result = pd.DataFrame(raw_result)
simulation_result.head()

In [ ]:
##########################
# 9. SIMULATION ANALYSIS #
##########################

# Get the agents network at the end
result = simulation_result.agents_network.iloc[-1]

alive = 0
dead = 0
for agent in result.nodes(data=True):
    if agent[1]['alive']:
        alive += 1
    else:
        dead += 1

print("count alive: " + str(alive))
print("count dead: " + str(dead))

# print("")
# print("result:")
# print(result.nodes(data=True))

In [ ]:
###############################
# 9. SIMULATION ANALYSIS PLOT #
###############################

dead = []
alive = []
infected = []
timestep = []
for agent_network_timestep in simulation_result.agents_network:
    died = 0
    survived = 0
    positive = 0
    for agent in agent_network_timestep.nodes(data=True):
        if agent[1]['infected']:
            positive += 1
        if agent[1]['alive']:
            survived += 1
        else:
            died += 1
    timestep.append(len(dead))
    dead.append(died)
    alive.append(survived)
    infected.append(positive)

losses_result = pd.DataFrame({"dead": dead, "alive": alive, "infected": infected, "timestep": timestep})

# print(losses_result)

pd.options.plotting.backend = "plotly"
losses_result.plot(
    kind='line',
    x='timestep',
    y=['infected', 'dead', 'alive']
)


---

---

# 3. System Validation

<center><img src="images/validation.png" alt="Validation" width="70%"/>